In [60]:
import os
import keras
import tensorflow as tf
from keras import layers
from keras import callbacks
from keras.src.legacy.preprocessing.image import ImageDataGenerator

from IPython.display import clear_output
from torch.backends.mkl import verbose

clear_output()

In [61]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [62]:
# Specify GPU usage
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [63]:
# Normalize image color values
datagen = ImageDataGenerator(rescale = 1.0 / 255.0)

# Specify path, target shape, batch size, classifier output, color mode, and shuffle
train_generator = datagen.flow_from_directory(
    '../data/train',
    target_size = (32, 32),
    batch_size = 32,
    class_mode = 'binary',
    color_mode = 'rgb',
    shuffle = True
)

test_generator = datagen.flow_from_directory(
    '../data/test',
    target_size = (32, 32),
    batch_size = 32,
    class_mode = 'binary',
    color_mode = 'rgb',
    shuffle = True
)

Found 20001 images belonging to 2 classes.
Found 4999 images belonging to 2 classes.


In [64]:
########### CODE PROVIDED BY PROFESSOR ###########
model = keras.Sequential([
    keras.Input(shape=(32, 32, 3)),  # Explicit Input Layer
    layers.Conv2D(32, (3, 3), activation="relu", padding="same"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation="relu", padding="same"),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),  # Dropout for regularization
    layers.Dense(1, activation="sigmoid")  # Binary classification output
])

In [65]:
########### CODE PROVIDED BY PROFESSOR ###########
model.compile(
	optimizer="adam",
    loss="binary_crossentropy",
	metrics=["accuracy"]
)

In [66]:
# Setup callbacks
early_stopping = callbacks.EarlyStopping(monitor = "val_loss", patience = 5, restore_best_weights = True)
model_checkpoint = callbacks.ModelCheckpoint('best_model.h5', monitor = 'val_loss', save_best_only = True)

In [67]:
model.fit(
    train_generator,
    epochs = 10,
    callbacks = [early_stopping , model_checkpoint],
    validation_data = test_generator,
    verbose = 2
)

Epoch 1/10


/Users/langtowl/miniconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


626/626 - 22s - 35ms/step - accuracy: 0.6086 - loss: 0.6501 - val_accuracy: 0.6873 - val_loss: 0.5790
Epoch 2/10


626/626 - 18s - 28ms/step - accuracy: 0.6874 - loss: 0.5921 - val_accuracy: 0.7425 - val_loss: 0.5273
Epoch 3/10


626/626 - 17s - 28ms/step - accuracy: 0.7429 - loss: 0.5231 - val_accuracy: 0.7642 - val_loss: 0.4986
Epoch 4/10


626/626 - 17s - 27ms/step - accuracy: 0.7672 - loss: 0.4865 - val_accuracy: 0.7728 - val_loss: 0.4807
Epoch 5/10


626/626 - 17s - 27ms/step - accuracy: 0.7858 - loss: 0.4550 - val_accuracy: 0.7770 - val_loss: 0.4771
Epoch 6/10


626/626 - 17s - 27ms/step - accuracy: 0.8019 - loss: 0.4297 - val_accuracy: 0.7850 - val_loss: 0.4610
Epoch 7/10


626/626 - 17s - 27ms/step - accuracy: 0.8163 - loss: 0.4076 - val_accuracy: 0.8006 - val_loss: 0.4312
Epoch 8/10
626/626 - 17s - 28ms/step - accuracy: 0.8271 - loss: 0.3841 - val_accuracy: 0.7902 - val_loss: 0.4498
Epoch 9/10
626/626 - 17s - 28ms/step - accuracy: 0.8367 - loss: 0.3696 - val_accuracy: 0.7806 - val_loss: 0.4826
Epoch 10/10


626/626 - 17s - 27ms/step - accuracy: 0.8499 - loss: 0.3464 - val_accuracy: 0.8090 - val_loss: 0.4226


In [68]:
########### CODE PROVIDED BY PROVESSOR ###########
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │     1,048,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,426,245 (13.07 MB)

 Trainable params: 1,142,081 (4.36 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,284,164 (8.71 MB)

In [70]:
# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, verbose=2)

print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_accuracy}")

157/157 - 4s - 22ms/step - accuracy: 0.8090 - loss: 0.4226
Test loss: 0.4225715398788452
Test accuracy: 0.808961808681488
